In [1]:
import requests as r
from bs4 import BeautifulSoup
import re
from collections import Counter
import threading
import time


In [2]:
def getLink(page): #把搜尋頁丟近來抓連結的function
    print('thread'+str(page)+'start') #測試多執行緒有沒有在動
    res = r.get("https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=104_bank1&ro=0&area=6001001000%2C6001002000&indcat=1001001000&order=2&asc=0&page={}".format(page)+"&psl=N_A")
    soup = BeautifulSoup(res.text,'lxml')
    links = ['https://www.104.com.tw' + link['href'] for link in soup.select('div.jobname_summary.job_name > a')]
    print(len(links))
    global alinks #拿到外面alinks的值如果沒有加global會變成自己增加區域變數
    alinks += links #alinks = alinks + links 

#     print('thread '+str(page)+' end ')    

In [3]:
class getLinkThread (threading.Thread): #多線程處理
    def __init__(self,page): #建構子（可以用來傳遞參數）（ex.我要傳入number這個參數讓每一條執行序可以跑不同頁
        threading.Thread.__init__(self) #繼承父類別（照著打就好）
        self.page=page #this.page=page
    def run(self): #等同於java執行緒中的run 把它overwrite 
        getLink(self.page) #將要爬的頁數丟給getLink去執行


In [4]:
def getWord(link): #把網址丟進來提取文字後做統計的function
    print(link)
    res = r.get(link)
    soup = BeautifulSoup(res.text,'lxml')
    list1 = str(soup.select('dd.tool > a')).upper() #擅長工具
    list2 = str(soup.select('div.content')).upper() #內文
    list3 = list1 + list2 
    words = list(set(re.findall('[A-Z]+[+#]*' , list3)))#擅長工具＋內文 中的所有英文單字
#     time.sleep(0.2)

    for word in words: # 計算所有英文單字的數量
        global wc
        if word in wc:
            wc[word] += 1
        else:
            wc[word] = 1    

In [5]:
class getWordThread (threading.Thread): #多線程處理
    def __init__(self,link): #建構子（可以用來傳遞參數）（ex.我要傳入number這個參數讓每一條執行序可以跑不同頁
        threading.Thread.__init__(self)
        self.link=link
    def run(self): #等同於java執行緒中的run 把它overwrite 
        getWord(self.link)


In [6]:
#第一頁開始：
url='https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=104_bank1&ro=0&jobcat=2007000000&order=2&asc=0&page=1'
res = r.get(url)
soup = BeautifulSoup(res.text,'lxml')
links = soup.select('div.jobname_summary.job_name > a') #裡面是沒有HOST的
HOST = 'https://www.104.com.tw'
link_list = [HOST + link['href'] for link in links] #把裡面每一個網址都加上HOST
#把link全加到link_list裡面


# 換頁：
buttons = soup.select('span > a')
first_page = int(buttons[3]['onclick'].split('gopage(')[1].split(');return')[0]) - 2 #3-2
#顯示當前頁面＋當頁所以職缺(一次爬?頁)
#把當頁所有職缺的網址放到alink裡面
alinks = []
page_to_crawl = 150  #要爬幾頁
threads=[]
for page in range(first_page, first_page + page_to_crawl , +1):
    Thread=getLinkThread(page)
    threads.append(Thread)    
for i in threads:
    i.start() #執行緒開始
#     time.sleep(1)
for i in threads:
    i.join()
    
#     print("current page {}".format("https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=104_bank1&ro=0&area=6001001000%2C6001002000&indcat=1001001000&order=2&asc=0&page=".format(page)+"&psl=N_A"))
#     res = r.get("https://www.104.com.tw/jobbank/joblist/joblist.cfm?jobsource=104_bank1&ro=0&area=6001001000%2C6001002000&indcat=1001001000&order=2&asc=0&page={}".format(page)+"&psl=N_A")
#     soup = BeautifulSoup(res.text,'lxml')
#     links = ['https://www.104.com.tw' + link['href'] for link in soup.select('div.jobname_summary.job_name > a')]
#     print(len(links))
#     alinks += links #alinks = alinks + links
#     time.sleep(0.5)


# for i in range(1,10+1): 
#     Thread=MyClass(i) 
#     threads.append(Thread)
# for thread in threads:
#     thread.start() #執行緒開始
# for thread in threads:
#     thread.join()#等同於java中的join 所有執行序跑完在繼續執行下一行指令
#     getLink(page)



#小整合：
wc = Counter() # word count   wc是k:v的形式-> ‘MYSQL': 4’
result_dict = {}
threadwords=[]
for link in alinks:
    threadword=getWordThread(link) 
    threadwords.append(threadword)
for i in threadwords:
    i.start() #執行緒開始
#     time.sleep(0.5)
for i in threadwords:
    i.join()
    
#         res = r.get(link)
#         soup = BeautifulSoup(res.text,'lxml')
#         list1 = str(soup.select('dd.tool > a')).upper() #擅長工具
#         list2 = str(soup.select('div.content')).upper() #內文
#         list3 = list1 + list2
#         words = list(set(re.findall('[A-Z]+[+#]*' , list3)))#擅長工具＋內文 中的所有英文單字

#         for word in words: # 計算所有英文單字的數量
#             if word in wc:
#                 wc[word] += 1
#             else:
#                 wc[word] = 1


#     continue
#     print(link) #有一些廣告或外包網continue
        
string = 'C|C++|C#|PYTHON|JAVA|JAVASCRIPT|PHP|HTML|SQL|CSS|R|BASH|RUBY|PERL|SCALA|SWIFT|GO|DELPHI|TYPESCRIPT|MYSQL'
langs = string.split('|') #string裡面的語言切開

for lang in langs: #每一個語言
    if lang in wc.keys(): #如果有在wc的key裡的話
        result_dict[lang] = wc[lang] #lang是key ;wc[lang]語言數量是value

result_dict

thread1start
thread2start
thread3start
thread4start
thread5start
thread6start
thread7start
thread8start
thread9start
thread10start
thread11start
thread12start
thread13start
thread14start
thread15start
thread16start
thread17start
thread18start
thread19start
thread20startthread21start

thread22start
thread23start
thread24start
thread25start
thread26start
thread27start
thread28start
thread29start
thread31start
thread30startthread32start

thread34startthread35startthread33start


thread36start
thread37startthread38start

thread39startthread40start
thread42startthread41start

thread43start

thread44start
thread45start
thread46start
thread47start
thread48start
thread49start
thread50start
thread52start
thread53start
thread54startthread51start

thread55startthread56start
thread57start
thread58start

thread59start
thread60start
thread61start
thread62start
thread64startthread63start

thread65startthread66start

thread67start
thread68start
thread70startthread69startthread71start


thread72start
t

https://www.104.com.tw/job/?jobno=5bdgk&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5bdkz&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4f6aq&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=49j07&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=5a19d&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3z0q7&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3zmty&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3oaps&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3q7ff&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=3tsk5&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=56yoh&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=56yor&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=57d6u&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=57hyw&jobsource=104_bank1&hotj

https://www.104.com.tw/job/?jobno=3ynel&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=59sqe&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=41n6j&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=3w80f&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=3fy26&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=3w80j&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=3wdmc&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=3xbum&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=tgj2&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=3w80h&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=3lc5g&jobsource=104_bank1&hotjob_chr=










https://www.104.com.tw/job/?jobno=5d32z&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5glfg&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5elzt&jobsource=104_bank1&hotjo

https://www.104.com.tw/job/?jobno=59p5q&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5a7q3&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=56hvv&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=56i05&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=56qan&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=56i0o&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=56i1u&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=5dzy7&jobsource=104_bank1&hotjob_chr=



https://www.104.com.tw/job/?jobno=555ng&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=525u3&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4vfrx&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4qb1o&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4mu0b&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5f1qj&jobsource=104_bank1&hotjo

https://www.104.com.tw/job/?jobno=3ses6&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3sexp&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=37b68&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=37b9v&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=2lr4x&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=2nkr6&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=2z0oi&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=31n66&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=321h1&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=54tlz&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4tfox&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3s0m5&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=354vz&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4v545&jobsource=104_bank1&hotj

...

https://www.104.com.tw/job/?jobno=4dsi6&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=523yu&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4e3xv&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4ftuu&jobsource=104_bank1&hotjob_chr=


https://www.104.com.tw/job/?jobno=5d7od&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4lly8&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=53aem&jobsource=104_bank1&hotjob_chr=


https://www.104.com.tw/job/?jobno=3vlz2&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4gyyq&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4g0no&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4lly9&jobsource=104_bank1&hotjob_chr=



https://www.104.com.tw/job/?jobno=3w278&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3m41f&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=3ceox&jobsource=104_bank1&hotj

Exception in thread Thread-1273:
Traceback (most recent call last):
  File "/home/skydream37/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/models.py", line 374, in prepare_url
    scheme, auth, host, port, path, query, fragment = parse_url(url)
  File "/home/skydream37/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/util/url.py", line 195, in parse_url
    raise LocationParseError(url)
requests.packages.urllib3.exceptions.LocationParseError: Failed to parse: www.104.com.twjavascript:void(0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/skydream37/.pyenv/versions/3.6.1/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "<ipython-input-5-cf618ee6c498>", line 6, in run
    getWord(self.link)
  File "<ipython-input-4-f82c11e50aef>", line 3, in getWord
    res = r.get(link)
  File "/home/skydream37/.pyenv/versions/3.6.1/lib/python3.6/site-packages/r

https://www.104.com.tw/job/?jobno=5g6xd&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5g7hb&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5g5vo&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=5cmoa&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=5cdzg&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=585ze&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=58ct5&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=54tcs&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=54tc9&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4exzx&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=54td4&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4u8hg&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4n2ks&jobsource=104_bank1&hotjob_chr=




https://www.104.com.tw/job/?jobno=4vkly&jobsource=104_bank1&hotj

https://www.104.com.tw/job/?jobno=55p4s&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=55p7a&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=56nh2&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4smgx&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4v1oa&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4vbhk&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=4vbhp&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4nyek&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=4ql5q&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=52rlm&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=51rv0&jobsource=104_bank1&hotjob_chr=

https://www.104.com.tw/job/?jobno=4snzz&jobsource=104_bank1&hotjob_chr=
https://www.104.com.tw/job/?jobno=47o1t&jobsource=104_bank1&hotjob_chr=https://www.104.com.tw/job/?jobno=4nljh&jobsource=104_bank1&hotjo

{'BASH': 5,
 'C': 174,
 'C#': 87,
 'C++': 85,
 'CSS': 139,
 'DELPHI': 1,
 'GO': 2,
 'HTML': 199,
 'JAVA': 152,
 'JAVASCRIPT': 387,
 'MYSQL': 101,
 'PERL': 7,
 'PHP': 112,
 'PYTHON': 46,
 'R': 15,
 'RUBY': 9,
 'SCALA': 9,
 'SQL': 133,
 'SWIFT': 16,
 'TYPESCRIPT': 5}